In [1]:
import os
from os import listdir
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import sys
sys.path.append("../../src")
from scipy.fft import fft
from FeaturesExtractorFromTSD import create_feature_dict_from_tsd
import matplotlib.pyplot as plt
import statistics

In [2]:
path_tds_angles = '../../results/big_dataset/TSD_angles'
folders = [x[0] for x in os.walk(path_tds_angles)]
folders.remove(path_tds_angles)

In [3]:
paths_to_csv = {}

In [4]:
for folder in folders:
    video_name = folder.split('\\')[-1]
    curr_paths_to_csv = [(f[:-4], os.path.join(folder, f)) for f in listdir(folder)]
    paths_to_csv[video_name] = curr_paths_to_csv

In [46]:
names_of_videos = [v for v in paths_to_csv.keys() if v in ['S007I', 'S005I', 'S009I', 'S002S']]
names_of_videos.sort(key=lambda v: v[::-1])

In [47]:
names_of_videos

['S001C',
 'S002C',
 'S007C',
 'S008C',
 'S009C',
 'S002I',
 'S003I',
 'S005I',
 'S007I',
 'S009I',
 'S001S',
 'S002S',
 'S004S',
 'S007S',
 'S009S']

In [59]:
len(names_of_videos)

15

In [7]:
def smooth_time_series(time_series, window_size):
    smoothed_series = []
    for i in range(len(time_series)):
        lower_bound = max(0, i - window_size)
        upper_bound = min(len(time_series), i + window_size + 1)
        window = time_series[lower_bound:upper_bound]
        average = sum(window) / len(window)
        smoothed_series.append(average)
    return smoothed_series

In [72]:
def calculate_sign_change_rate(time_series):
    count = 0
    for i in range(1, len(time_series)):
        if time_series[i] * time_series[i-1] < 0:
            count += 1
    #sign_change_rate = count / (len(time_series) - 1)
    return count

def calc_area(values):
  values = np.array(values)
  indices = np.where(values > 0)[0]
  area_pos = np.trapz(values[indices])
  indices = np.where(values < 0)[0]
  area_neg = np.trapz(values[indices])
  return area_pos, area_neg

def shift_data(series):
  median = statistics.median(series)
  shifted_data = [x - median for x in series]
  return shifted_data

def classify_time_series(values, titles):
  arr = []

  for v in values:
    s = float(calculate_sign_change_rate(v))
    arr.append(s)

  thres= float(statistics.mean(arr))

  for i, series in enumerate(values):
    s = arr[i]
    title = titles[i]

    if s > thres:
      print(f"s:{s}, thres:{thres}")
      print(title, " Shuffled")
    else:
      print(f"s:{s}, thres:{thres}")
      a1, a2 = calc_area(series)
      if abs(a2) < a1/3:
        print(title," Inverted")
      else:
        print(title," Correct")


In [9]:
experiment_suff = 'classif_wrong_pred'

In [145]:

def classify_time_series(angl_bow_mov, titles):
    num_rows = 8
    num_cols = 2
    shifted_smooth_series = []
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(50, 18))
    plt.rcParams['font.size'] = 20
    axes = axes.flatten()
    sign_change_arr = []
    #print(len(angl_bow_mov))
    for i, series in enumerate(angl_bow_mov):
      smoothed_data = smooth_time_series(series, 10)
      #neg_smoothed_data = negate_below_median(smoothed_data)
      title = titles[i]
      #num_sign_change = calculate_sign_change_rate(neg_smoothed_data)
      #print(title, " ", num_sign_change)

      ax = axes[i]
        #series = eliminate_lowest_values(series,10)
        #average = sum(time_series) / len(time_series)
      median = statistics.median(smoothed_data)
      #mode = statistics.mode(smoothed_data)
      shifted_data = shift_data(smoothed_data)
      #shifted_data = smooth_time_series(shifted_data, 15)
      ax.axhline(median, color='r', linestyle='--', label='median')
      ax.axhline(0, color='g', linestyle='--', label='median')
      #smoothed_data = smooth_time_series(series, 10) #np.convolve(series, np.ones(window_size) / window_size, mode='valid')
      ax.plot(shifted_data, label='Shifted Data')
      ax.plot(smoothed_data, label='Smoothed Data')
      #ax.set_title(title)
      #ax.legend()
      #smooth_series.append(smoothed_data)
      s =calculate_sign_change_rate(shifted_data)
      a1, a2 = calc_area(shifted_data)
      #print(title, s, a1,a2, a1+a2)
      sign_change_arr.append(s)
      shifted_smooth_series.append(shifted_data)

    arr = []

    for v in shifted_smooth_series:
        s = calculate_sign_change_rate(v)
        arr.append(s)

    thres = int(statistics.mean(arr))

    for i, series in enumerate(shifted_smooth_series):
        s = arr[i]
        title = titles[i]
        ax = axes[i]
        print(f"s:{s}, thres:{thres}")
        if s > thres:
          title += f" Shuffled - {str('S'==title[-1])}"
        else:
          a1, a2 = calc_area(series)
          if abs(a2) < a1/2:
            title += f" Inverted - {str('I'==title[-1])}"
          else:
            title += f" Correct - {str('C'==title[-1])}"

        print(title)
        print('------------------')
        ax.set_title(title)

    plt.tight_layout()
    plt.savefig(experiment_suff+"ts_classification.png")
    plt.close()
    #return shifted_smooth_series

In [146]:
bow_moving = []
titles = []

In [147]:
for i in range(len(names_of_videos)):
    curr_metric, curr_path = paths_to_csv[names_of_videos[i]][0]
    print(curr_path)
    print(curr_metric)
    title = curr_path.split('/')[-1].split('\\')[1]
    titles.append(title)
    print(title)
    angles = pd.read_csv(curr_path)['angle'].to_list()
    bow_moving.append(angles)

../../results/big_dataset/TSD_angles\S001C\angl_bow_mov.csv
angl_bow_mov
S001C
../../results/big_dataset/TSD_angles\S002C\angl_bow_mov.csv
angl_bow_mov
S002C
../../results/big_dataset/TSD_angles\S007C\angl_bow_mov.csv
angl_bow_mov
S007C
../../results/big_dataset/TSD_angles\S008C\angl_bow_mov.csv
angl_bow_mov
S008C
../../results/big_dataset/TSD_angles\S009C\angl_bow_mov.csv
angl_bow_mov
S009C
../../results/big_dataset/TSD_angles\S002I\angl_bow_mov.csv
angl_bow_mov
S002I
../../results/big_dataset/TSD_angles\S003I\angl_bow_mov.csv
angl_bow_mov
S003I
../../results/big_dataset/TSD_angles\S005I\angl_bow_mov.csv
angl_bow_mov
S005I
../../results/big_dataset/TSD_angles\S007I\angl_bow_mov.csv
angl_bow_mov
S007I
../../results/big_dataset/TSD_angles\S009I\angl_bow_mov.csv
angl_bow_mov
S009I
../../results/big_dataset/TSD_angles\S001S\angl_bow_mov.csv
angl_bow_mov
S001S
../../results/big_dataset/TSD_angles\S002S\angl_bow_mov.csv
angl_bow_mov
S002S
../../results/big_dataset/TSD_angles\S004S\angl_bow_

In [148]:
classify_time_series(bow_moving, titles)

s:119, thres:149
S001C Correct - True
------------------
s:105, thres:149
S002C Correct - True
------------------
s:107, thres:149
S007C Correct - True
------------------
s:96, thres:149
S008C Correct - True
------------------
s:102, thres:149
S009C Correct - True
------------------
s:146, thres:149
S002I Inverted - True
------------------
s:127, thres:149
S003I Inverted - True
------------------
s:154, thres:149
S005I Shuffled - False
------------------
s:110, thres:149
S007I Inverted - True
------------------
s:116, thres:149
S009I Inverted - True
------------------
s:242, thres:149
S001S Shuffled - True
------------------
s:120, thres:149
S002S Correct - False
------------------
s:242, thres:149
S004S Shuffled - True
------------------
s:262, thres:149
S007S Shuffled - True
------------------
s:197, thres:149
S009S Shuffled - True
------------------


## prepare data

In [149]:
titles

['S001C',
 'S002C',
 'S007C',
 'S008C',
 'S009C',
 'S002I',
 'S003I',
 'S005I',
 'S007I',
 'S009I',
 'S001S',
 'S002S',
 'S004S',
 'S007S',
 'S009S']

In [150]:
len(bow_moving)

15

In [175]:
bow_moving_prepared = bow_moving.copy()

In [176]:
print(titles[7])
bow_moving_prepared[7] = bow_moving_prepared[7][6000:]
print(titles[11])
bow_moving_prepared[11] = bow_moving_prepared[11][100:]

S005I
S002S


In [177]:
print(titles[5])
bow_moving_prepared[5] = bow_moving_prepared[5][500:-500]

S002I


In [178]:
classify_time_series(bow_moving_prepared, titles)

s:119, thres:145
S001C Correct - True
------------------
s:105, thres:145
S002C Correct - True
------------------
s:107, thres:145
S007C Correct - True
------------------
s:96, thres:145
S008C Correct - True
------------------
s:102, thres:145
S009C Correct - True
------------------
s:137, thres:145
S002I Inverted - True
------------------
s:127, thres:145
S003I Inverted - True
------------------
s:99, thres:145
S005I Inverted - True
------------------
s:110, thres:145
S007I Inverted - True
------------------
s:116, thres:145
S009I Inverted - True
------------------
s:242, thres:145
S001S Shuffled - True
------------------
s:122, thres:145
S002S Correct - False
------------------
s:242, thres:145
S004S Shuffled - True
------------------
s:262, thres:145
S007S Shuffled - True
------------------
s:197, thres:145
S009S Shuffled - True
------------------
